# Cortes
>$n$: Quantidade de tipos de itens  
>$m$: Quantidade de padrões de corte  

## Variáveis
>$x_j$: Quantidade que será utilizada do padrão de corte $j$, para $j = 1,\ldots, m$  

## Parâmetros
>$D_i$: Demanda do item $i$, para $i = 1,\ldots, n$  
>$P_{ij}$: Padrão de corte $i$ do tipo de item $j$, para $i = 1,\ldots, n$ e $j = 1,\ldots, m$  

## Modelo  

\begin{align*}
    \hbox{min} \ \ 
        & \sum_{j=1}^m x_i \\
        
    \hbox{s.a.} \ \ 
        & \sum_{j=1}^m P_{ij} x_j \geq D_i && i = 1,\ldots, n \\
        & x_{j} \in +\mathbb{Z}            && j = 1,\ldots, m \\
\end{align*}

In [7]:
from itertools import product


def cut_patterns(M: int, C: list[int]) -> list[list[int]]:
    corte_minimo = M - min(C)
    cortes = product(*(range(M//i + 1) for i in C))
    x = [i for i in cortes if corte_minimo < sum(a*b for a, b in zip(i, C)) <= M]

    return x

In [8]:
import pandas as pd
from IPython.display import display

def create_table(M: int, C: list[int]):
    patterns = cut_patterns(M, C)
    df = pd.DataFrame(patterns, columns=C)
    df["Resto"] = M - df.mul(C).sum(axis=1)
    # display(df)
    
    return df.to_string()

In [9]:
from pymprog import *


def modelo_cortes(M, C, D):
    P = [*zip(*cut_patterns(M, C))]

    n = len(D)
    m = len(P[0])

    begin("cortes")

    x = var("x", m, int)

    minimize( sum(x) )

    for i in range(n):
        sum(P[i][j] * x[j] for j in range(m)) >= D[i]

    # solver(int, tm_lmt= 60_000)
    solve()

    obj = vobj()

    end()

    return obj, x, P

In [10]:
import os
import pandas as pd

resumo = pd.DataFrame(columns=[
    "Tamanho da matriz",
    "Tamanho dos itens menores",
    "Demanda dos itens menores",
    "Qtn. de padrões de corte gerados",
    "Qtn. de padrões de corte utilizados",
    "Total de papel usado",
    "Qtn. de papel desperdiçada"
])

for file in os.listdir("./input/"):
    with open(f"input/{file}", "r") as f:
        M = int(f.readline())

        # descarta dado não utilizado
        f.readline()

        C, D = [], []
        for i in f.readlines():
            c, d = map(int, i.split())
            C.append(c)
            D.append(d)

        print(f"\n{file:=^70}")

        # create_table(M, C)
        obj, varx, P = modelo_cortes(M, C, D)
        P = [*zip(*P)]

        with open(f"output_a/{file}", "w") as f:
            f.write(create_table(M, C) + "\n\n")

            f.write(f"Tamanho da bobina matriz: {M}\n")
            f.write(f"Tamanho das bobinas menores: {C}\n")
            f.write(f"Demandas: {D}\n\n")

            f.write(f"Quantidade de bobinas usadas: {int(obj)}\n\n")

            desperdicio_total = 0

            for i, x in enumerate(varx):
                if x.primal:
                    resto = M - sum(a*b for a, b in zip(P[i], C))
                    desperdicio_total += int(x.primal) * resto

                    f.write(f"Padrão {i:>2}: {P[i]} resto {resto}\n")
                    f.write(f"Quantidade usada: {int(x.primal)}\n")
                    f.write(f"Quantidade desperdiçada: {int(x.primal) * resto}\n")
                    f.write(f"Quantidade satisfeita de cada demanda: {tuple(j * int(x.primal) for j in P[i])}\n\n")

            f.write(f"Quantidade total de papel gasto: {int(obj) * M}\n")
            f.write(f"Quantidade desperdiçada: {desperdicio_total}\n")

            resumo.loc[file] = [
                M,
                C,
                D,
                len(cut_patterns(M, C)),
                sum(bool(x.primal) for x in varx),
                int(obj) * M,
                desperdicio_total
            ]
            


============================inst_200_5.txt============================
GLPK Simplex Optimizer 5.0
5 rows, 51 columns, 129 non-zeros
      0: obj =   0.000000000e+00 inf =   1.209e+04 (5)
      5: obj =   3.446229167e+03 inf =   0.000e+00 (0)
*     7: obj =   3.338166667e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Integer Optimizer 5.0
5 rows, 51 columns, 129 non-zeros
51 integer variables, none of which are binary
Integer optimization begins...
Long-step dual simplex will be used
+     7: mip =     not found yet >=              -inf        (1; 0)
Solution found by heuristic: 3339
+    10: mip =   3.339000000e+03 >=     tree is empty   0.0% (0; 7)
INTEGER OPTIMAL SOLUTION FOUND

============================inst_130_6.txt============================
GLPK Simplex Optimizer 5.0
6 rows, 48 columns, 113 non-zeros
      0: obj =   0.000000000e+00 inf =   1.525e+04 (6)
      6: obj =   5.202175000e+03 inf =   0.000e+00 (0)
*    10: obj =   5.134708333e+03 inf =   0.000e+00 (0)
OP

In [11]:
display(
    resumo.style.set_properties(
        subset=[
            "Qtn. de padrões de corte gerados",
            "Qtn. de padrões de corte utilizados",
            "Total de papel usado",
            "Qtn. de papel desperdiçada",
        ],
        width="130px",
    )
)


,Tamanho da matriz,Tamanho dos itens menores,Demanda dos itens menores,Qtn. de padrões de corte gerados,Qtn. de padrões de corte utilizados,Total de papel usado,Qtn. de papel desperdiçada
inst_200_5.txt,200,"[21, 24, 43, 79, 98]","[1876, 3448, 1142, 3896, 1731]",51,8,667800,18957
inst_130_6.txt,130,"[19, 22, 28, 54, 63, 64]","[3050, 2334, 1256, 3213, 2066, 3335]",48,6,667550,5946
inst_200_4.txt,200,"[21, 43, 79, 98]","[1876, 3448, 1142, 3896]",14,3,676200,13826
inst_110_5.txt,110,"[10, 12, 17, 18, 21]","[1320, 1560, 2210, 1000, 1480]",229,9,118580,0
inst_100_4.txt,100,"[15, 16, 31, 40]","[1040, 3220, 3207, 1048]",24,4,219400,10943
inst_250_5.txt,250,"[33, 43, 52, 60, 87]","[3206, 1135, 2287, 3304, 3286]",64,6,769500,11698
inst_100_6.txt,100,"[15, 16, 18, 31, 32, 40]","[1040, 3220, 3207, 1048, 1370, 2637]",91,7,312800,6106
inst_130_4.txt,130,"[19, 28, 54, 63]","[3050, 2334, 1256, 3213]",14,5,400530,6938
inst_250_6.txt,250,"[33, 43, 52, 53, 60, 87]","[3206, 1135, 2287, 3304, 3286, 1420]",119,8,772500,3002
inst_200_6.txt,200,"[21, 24, 43, 53, 79, 98]","[1876, 3448, 1142, 3896, 1731, 2923]",87,6,816800,15757
